<a href="https://colab.research.google.com/github/bposantos/nmr/blob/main/T1_relaxation_peptide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T1 relaxation curve fitting

## Data importing and preprocessing

In [ ]:
#Necessary libraries
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import matplotlib.pyplot as plt
import string
import numpy as np

In [ ]:
# User authentication
auth.authenticate_user()

# Credentials
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# File openning
sheet_name = 't1_peptide'
sheet = gc.open(sheet_name)

# Choosing the pages of the file
page_name1 = 'pep1'
page1 = sheet.worksheet(page_name1)
page_name2 = 'pep2'
page2 = sheet.worksheet(page_name2)
page_name3 = 'pep3'
page3 = sheet.worksheet(page_name3)
page_name4 = 'pep4'
page4 = sheet.worksheet(page_name4)
page_name5 = 'pep5'
page5 = sheet.worksheet(page_name5)
page_name6 = 'pep6'
page6 = sheet.worksheet(page_name6)

In [ ]:
# Obtaining data values
data1 = page1.get_all_values()
data2 = page2.get_all_values()
data3 = page3.get_all_values()
data4 = page4.get_all_values()
data5 = page5.get_all_values()
data6 = page6.get_all_values()

In [ ]:
# Pandas dataframes
df1 = pd.DataFrame(data1[1:], columns=data1[0])
df2 = pd.DataFrame(data2[1:], columns=data2[0])
df3 = pd.DataFrame(data3[1:], columns=data3[0])
df4 = pd.DataFrame(data4[1:], columns=data4[0])
df5 = pd.DataFrame(data5[1:], columns=data5[0])
df6 = pd.DataFrame(data6[1:], columns=data6[0])

In [ ]:
# Dataframes list
dataframes = [df1, df2, df3, df4, df5, df6]

In [ ]:
# Excluding undersired rows
row_to_exclude = [0,1]

for i, df in enumerate(dataframes,start=1):
    # Filtering rows with the excluded index
    df.drop(df.index[row_to_exclude], inplace=True)
    print(f"The dataframe {i} has been adjusted.")

In [ ]:
# Converting the intensity column to numeric
for i,df in enumerate(dataframes,start=1):
  df['Intensity'] = pd.to_numeric(df['Intensity'])

## Curve Fitting

In [ ]:
# Libraries
import numpy as np
from scipy.optimize import curve_fit
from matplotlib import gridspec
import string

In [ ]:
# Exponential function with a negative exponential term (T1 relaxation curve)
"""
Determining the T1 relaxation time involves fitting experimental data to a suitable relaxation model.
T1 relaxation is a characteristic time constant that describes the time it takes for a nuclear spin system
to return to equilibrium after being perturbed.

The most common model for T1 relaxation is an exponential recovery model, often expressed as:
Mz(t) = M0 . (1 - 2 . e^(-t/T1))

where Mz(t) is the magnetization at time t, M0 is the initial magnetization, T1 is the T1 relaxation time constant, and x(t) is the time-dependent, with e being the base of the natural logarithm.
"""
def func(x,a,b):
    return a * (1 - 2 * np.exp(-x / b))

In [ ]:
# Seaborn importing
import seaborn as sns
sns.set_theme(style="white")
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
sns.set_context("paper", font_scale=1.5)

In [ ]:
# Setting style and pallete
print(plt.style.available)
plt.style.use('seaborn-v0_8-paper')

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [ ]:
# Converting the time column to numeric
for i,df in enumerate(dataframes,start=1):
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')

  # Normalization
  df.iloc[:,0] = df.iloc[:,0]/(df.iloc[:,0]).max()
  df.iloc[:,1] = df.iloc[:,1]/max(df.iloc[:,1])

<ipython-input-16-c6b9f18b84fe>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')


## Plotting

In [ ]:
from scipy.optimize import curve_fit

### T1 curves

In [ ]:
# Subplotting grid size
num_rows = 3
num_cols = 3

# Subplotting creation
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

# Subplots unpacking
axs = axs.ravel()

# DataFrames iteration
for i, df in enumerate(dataframes):
    xdata = df['Time (ms)'].astype(float).values
    ydata = df['Intensity'].values

    popt, pcov = curve_fit(func, xdata, ydata)
    print(f"O valor do dataframe {i + 1} é: {popt[0]:.3f} e {popt[1]:.3f}")

    # Índices do subplot na grade
    row = i // num_cols
    col = i % num_cols

    # Plotagem nos subplots
    axs[i].plot(xdata, ydata, 'bo', label='data')
    axs[i].plot(xdata, func(xdata, *popt), 'r-', label='fit')
    axs[i].legend(loc='best')
    #axs[i].set_title(f'DataFrame {i + 1}')
    axs[0].set_title(f'pep1')
    axs[1].set_title(f'pep2')
    axs[2].set_title(f'pep3')
    axs[3].set_title(f'pep4')
    axs[4].set_title(f'pep5')
    axs[5].set_title(f'pep6')
    axs[i].set_xlabel('Time (ms)')
    axs[i].set_ylabel('Signal (a.u.)')

    # Calculate R-squared
    residuals = ydata - func(xdata, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata - np.mean(xdata))**2)
    r_squared = 1 - (ss_res / ss_tot)

    # Text
    textstr = '\n'.join((
        r'$R^2=%.3f$' % (r_squared, ),
        f"I = {popt[0]:.3f} *(1 - 2 * e^(-x/{popt[1]:.3f}))"))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    axs[i].text(0.35, -0.55, textstr, fontsize=8, verticalalignment='top', bbox=props)

    # Exporting formulas
    y_formula = f"I = {popt[0]:.3f} *(1 - 2 * e^(-x/{popt[1]:.3f}))"

    # Assigning formulas to different variables
    globals()[f'df{i}_formula'] = y_formula

# Final adjustments
plt.suptitle('Lunatin1 analogues', fontsize=24, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### T1 values barplot

In [ ]:
# T1 values list
popt_values = []

# Peptide names list
dataset_labels=['pep1', 'pep2', 'pep3', 'pep4', 'pep5', 'pep6']

# DataFrames Iteration
for i, df in enumerate(dataframes):
    xdata = df['Time (ms)'].astype(float).values
    ydata = df['Intensity'].values

    popt, pcov = curve_fit(func, xdata, ydata)
    popt_values.append(popt[0])

# Figure resolution adjustment
%config InlineBackend.figure_format = 'retina'

# Barplot creation
sns.barplot(x=dataset_labels, y=popt_values, palette='rocket')
plt.xlabel('Peptides')
plt.ylabel('T1 (s)')
plt.title('T1 values')
plt.show()